In [ ]:
import numpy as np
from viz import quickplot
import matplotlib.pyplot as plt
import seaborn as sns
import mne
import pickle as pkl
import time
from sim import *
from inverse_solutions import *
from source_covs import *
from util import *
from evaluate import *
from par import *
from tqdm import tqdm
from joblib import Parallel, delayed
%matplotlib qt
pth_res = 'assets'

## Load Data

In [ ]:
with open(pth_res + '/leadfield.pkl', 'rb') as file:
    leadfield = pkl.load(file)[0]
with open(pth_res + '/pos.pkl', 'rb') as file:  
    pos = pkl.load(file)[0]
with open(pth_res + '/info.pkl', 'rb') as file:  
    info = pkl.load(file)

fwd = mne.read_forward_solution(pth_res + '/fsaverage-fwd.fif', verbose=0)
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)

## Region growing

In [ ]:
numberOfDipoles, numberOfElectrodes = pos.shape
tris_lr = [fwd['src'][0]['use_tris'], fwd['src'][1]['use_tris']]
pos_lr = [pos[:int(numberOfDipoles/2), :], pos[-int(numberOfDipoles/2):, :]]
# determine distance of closest neighbor for each dipole
distanceCrit = calc_dist_crit(pos)
print(f'A neighbor is a voxel in {distanceCrit:.1f} mm vicinity')
# Pick single dipole
dip_idx = np.random.choice(np.arange(numberOfDipoles), size=1)[0]
hem = int(pos[dip_idx, 0] > 0)
# Find its first neighbors on the mesh



In [ ]:
tris_lr[hem]

In [ ]:
idx = 55
np.where(tris_lr[0] ==idx)
tris_lr[0][np.where(tris_lr[0] ==idx)[0], :]

In [ ]:
import time
start = time.time()
for i in range(numberOfDipoles):
    tris_lr[0][np.where(tris_lr[0] ==1)[0], :]
end = time.time()
print(end-start)

In [ ]:
pos[tris_lr[hem][3], :]
# Make a list for each dipole with indices of its neighbors
neighbors = [[]] * numberOfDipoles
# left
for idx in range(numberOfDipoles):
    trianglesOfIndex = tris_lr[0][np.where(tris_lr[0] == idx)[0], :]
    # print(tri)
    for tri in trianglesOfIndex:
        neighbors[idx].extend(tri)
        neighbors[idx] = list(filter(lambda a: a != idx, neighbors[idx]))
        
    
# right
for i, tri in enumerate(tris_lr[1]):
    pass

In [ ]:
neighbors

In [ ]:
def calc_dist_vec(singlePosition, allPositions):
    return np.sqrt(np.sum(np.square(allPositions - singlePosition), axis=1))
def calc_dist_crit(pos):
    numberOfDipoles = pos.shape[0]
    distOfClosestNeighbor = np.zeros((numberOfDipoles))
    for i in range(numberOfDipoles):
        distVec = calc_dist_vec(pos[i, :], pos)
        distOfClosestNeighbor[i] = np.min(distVec[np.nonzero(distVec)])
    distanceCrit = np.ceil(np.max(distOfClosestNeighbor))
    return distanceCrit